<a href="https://colab.research.google.com/github/Jijzzz/Mycaptain/blob/master/text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import Dependencies

import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [3]:
#Import gdrive 

from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [4]:
#Load data

file = open("/content/mydrive/My Drive/Frankenstein-2.txt").read()
print(file)

We were brought up together; there was not quite a year difference in
our ages. I need not say that we were strangers to any species of
disunion or dispute. Harmony was the soul of our companionship, and the
diversity and contrast that subsisted in our characters drew us nearer
together. Elizabeth was of a calmer and more concentrated disposition;
but, with all my ardour, I was capable of a more intense application,
and was more deeply smitten with the thirst for knowledge. She busied
herself with following the aerial creations of the poets; and in the
majestic and wondrous scenes which surrounded our Swiss home--the
sublime shapes of the mountains; the changes of the seasons; tempest and
calm; the silence of winter, and the life and turbulence of our Alpine
summers,--she found ample scope for admiration and delight. While my
companion contemplated with a serious and satisfied spirit the
magnificent appearances of things, I delighted in investigating their
causes. The world was to me a

In [5]:
#Tokenization
#Standardization

def tokenize_words(input):
  global file
  input = input.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  print(tokens)
  filtered = list(filter(lambda token: token not in stopwords.words('english'), tokens))
  print(filtered)
  return "".join(filtered)

  
processed_inputs = tokenize_words(file)

['we', 'were', 'brought', 'up', 'together', 'there', 'was', 'not', 'quite', 'a', 'year', 'difference', 'in', 'our', 'ages', 'i', 'need', 'not', 'say', 'that', 'we', 'were', 'strangers', 'to', 'any', 'species', 'of', 'disunion', 'or', 'dispute', 'harmony', 'was', 'the', 'soul', 'of', 'our', 'companionship', 'and', 'the', 'diversity', 'and', 'contrast', 'that', 'subsisted', 'in', 'our', 'characters', 'drew', 'us', 'nearer', 'together', 'elizabeth', 'was', 'of', 'a', 'calmer', 'and', 'more', 'concentrated', 'disposition', 'but', 'with', 'all', 'my', 'ardour', 'i', 'was', 'capable', 'of', 'a', 'more', 'intense', 'application', 'and', 'was', 'more', 'deeply', 'smitten', 'with', 'the', 'thirst', 'for', 'knowledge', 'she', 'busied', 'herself', 'with', 'following', 'the', 'aerial', 'creations', 'of', 'the', 'poets', 'and', 'in', 'the', 'majestic', 'and', 'wondrous', 'scenes', 'which', 'surrounded', 'our', 'swiss', 'home', 'the', 'sublime', 'shapes', 'of', 'the', 'mountains', 'the', 'changes', 

In [6]:
#Chars to no.s

chars = sorted(list(set(processed_inputs)))  
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [7]:
#Check if words to chars or chars to num (?!) has worked?

input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters:", input_len)
print("Total vocab:", vocab_len)
print(chars)

Total number of characters: 7036
Total vocab: 27
['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
#Seq length

seq_length = 100
x_data = []
y_data = []

In [9]:
#Loop through the sequence

for i in range(0, input_len - seq_length, 1):
  in_seq = processed_inputs[i:i + seq_length]
  out_seq = processed_inputs[i + seq_length]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 6936


In [10]:
#Convert input sequence to np and array and so on

X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float (vocab_len)

In [11]:
#One-hot encoding

y = np_utils.to_categorical(y_data)

In [12]:
#Creating the model

model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [13]:
#Compile the model

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
#Saving weights

filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [15]:
#Fit model and let it train

model.fit(X,y, epochs=4, batch_size=256, callbacks = desired_callbacks)

Epoch 1/4
6936/6936 [==============================] - 118s 17ms/step - loss: 2.9880

Epoch 00001: loss improved from inf to 2.98798, saving model to model_weights_saved.hdf5
Epoch 2/4
6936/6936 [==============================] - 117s 17ms/step - loss: 2.9183

Epoch 00002: loss improved from 2.98798 to 2.91835, saving model to model_weights_saved.hdf5
Epoch 3/4
6936/6936 [==============================] - 117s 17ms/step - loss: 2.9006

Epoch 00003: loss improved from 2.91835 to 2.90062, saving model to model_weights_saved.hdf5
Epoch 4/4
6936/6936 [==============================] - 117s 17ms/step - loss: 2.9037

Epoch 00004: loss did not improve from 2.90062


In [16]:
#Recompile model with the saved weights

filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer= 'adam')

In [17]:
#Output of the model back into characters

num_to_char = dict((i,c) for i,c in enumerate(chars))

In [18]:
#Random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\" ")

Random seed:
" yusexcitedcatastropheenteredexplanationtheoryformedsubjectelectricitygalvanismnewastonishingsaidthre " 


In [19]:
#Generate the text

for i in range(1000):
  x = numpy.reshape(pattern, (1,len(pattern), 1))
  x = x/float (vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee